In [1]:
import os
from PIL import Image
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
from utils import datasets
from utils import utils
from utils import train_val
from utils import net #网络文件于此
from utils import metrics
import warnings

# 完全禁用警告
warnings.filterwarnings("ignore")

In [2]:
from utils.utils import Config,Logs,BestSelector
config=utils.Config(
    dataset_sep=[
        0.82,0.17,0.01          
        ],
    resize_size=128,
    batch_size=128,
    lr=0.001,
    epochs=20,
    hidden_size=256,
    optim="Adam",
    momentum=0.9,
    weight_decay=1e-4,
    seed=42,
    mean= [0.50638 ,0.49962538 ,0.45205265],
    std=[0.23568255 ,0.24141274 ,0.25167742],
    checkpoint_interval=0.25,#只保存4个模型
    source_dir=r"Cifar-10",#原始数据集，每个分类一个文件夹，每个文件夹里包含多个图片
    data_path=r"data\Cifar-10",#项目数据集
    # classes=["Apple","Carambola","Pear","Plum","Tomatoes"],
    device="cuda" if torch.cuda.is_available() else "cpu",
)
print(config)

dataset_sep : [0.82, 0.17, 0.01] 
resize_size : 128 
batch_size : 128 
lr : 0.001 
epochs : 20 
hidden_size : 256 
optim : Adam 
momentum : 0.9 
weight_decay : 0.0001 
seed : 42 
mean : [0.50638, 0.49962538, 0.45205265] 
std : [0.23568255, 0.24141274, 0.25167742] 
checkpoint_interval : 0.25 
source_dir : Cifar-10 
data_path : data\Cifar-10 
device : cuda 


In [3]:
from tqdm import tqdm

#设置随机数种子
torch.manual_seed(config.seed)

isSplit= (not datasets.check_data_exist(config.data_path))#数据集不存在，则从原始数据存放处，转移数据集
if isSplit:
    print(f"{config.data_path} 数据集不存在，将从source_dir:{config.source_dir}中获取数据")
    print(f"清理源文件夹:{config.data_path}")
    datasets.clear_folder(config.data_path)
    for dir in tqdm(config.classes,desc="处理原始数据："):
        source_dir=os.path.join(config.source_dir, dir)
        print(source_dir)
        datasets.split_data(source_dir,target_dir=config.data_path,label=dir,sep=config.dataset_sep)
else:
    print(f"{config.data_path} 数据集已存在，无需重新划分")



data\Cifar-10 数据集已存在，无需重新划分


In [4]:
'''
def get_data_path(typ,path=config.data_path):
    return os.path.join(path,typ)
train_dataset=datasets.CustomImageDataset(
    get_data_path("train"),
    classes=config.classes,
    transform=datasets.get_transform(
        resize_size=config.resize_size,
        mean=config.mean,std=config.std
        )
)
val_dataset=datasets.CustomImageDataset(
    get_data_path("val"),
    classes=config.classes,
    transform=datasets.get_transform(
        "val",
        config.resize_size,
        
        mean=config.mean,
        std=config.std
        )
)
'''


'\ndef get_data_path(typ,path=config.data_path):\n    return os.path.join(path,typ)\ntrain_dataset=datasets.CustomImageDataset(\n    get_data_path("train"),\n    classes=config.classes,\n    transform=datasets.get_transform(\n        resize_size=config.resize_size,\n        mean=config.mean,std=config.std\n        )\n)\nval_dataset=datasets.CustomImageDataset(\n    get_data_path("val"),\n    classes=config.classes,\n    transform=datasets.get_transform(\n        "val",\n        config.resize_size,\n        \n        mean=config.mean,\n        std=config.std\n        )\n)\n'

#### 加载数据集，可本地数据集。调用datasets.CustomImageDataset

In [5]:
#加载cifar-10
import torchvision
train_val_dataset = torchvision.datasets.CIFAR10(
    root='./data/Cifar-10', 
    train=True, 
    download=True, 
    transform=datasets.get_transform(
        chance="train",
        resize_size=config.resize_size,
        mean=(0.5, 0.5, 0.5), 
        std=(0.5, 0.5, 0.5)
    )
)
train_dataset,val_dataset=datasets.get_parts_of_datasets(train_val_dataset,rate=0.75,only_train=False)#训练验证3-1开

test_dataset = torchvision.datasets.CIFAR10(root='./data/Cifar-10', train=False, download=True, transform=datasets.get_transform(
    chance="val",
    resize_size=config.resize_size,
    mean=(0.5, 0.5, 0.5), 
    std=(0.5, 0.5, 0.5)
))
print(f"原始训练集大小：{len(train_dataset)}")
print(f"原始验证集大小：{len(val_dataset)}")
print(f"原始测试集大小：{len(test_dataset)}")

Files already downloaded and verified
Files already downloaded and verified
原始训练集大小：37500
原始验证集大小：12500
原始测试集大小：10000


#### load数据集，并根据数据量进行裁剪

In [6]:
crop_rate=0.25
train_dataset_crop=datasets.get_parts_of_datasets(train_dataset,crop_rate)
val_dataset_crop=datasets.get_parts_of_datasets(val_dataset,crop_rate)
test_dataset_crop=datasets.get_parts_of_datasets(test_dataset,crop_rate)
train_dataset_size=len(train_dataset_crop)
val_dataset_size=len(val_dataset_crop)
test_dataset_size=len(test_dataset_crop)

print(f"本次训练用训练集大小：{len(train_dataset_crop)}")
print(f"本次训练用验证集大小：{len(val_dataset_crop)}")
print(f"本次测试用测试集大小：{len(test_dataset_crop)}")

train_loader=DataLoader(train_dataset_crop,batch_size=config.batch_size,shuffle=True,drop_last=True)
val_loader=DataLoader(val_dataset_crop,batch_size=config.batch_size,shuffle=False)
test_loader=DataLoader(test_dataset_crop,batch_size=config.batch_size,shuffle=False)

print(f"tarin_dataloader加载完毕, {len(train_loader)}个batch, batch大小为{config.batch_size}")
print(f"val_dataloader  加载完毕, {len(val_loader)}个batch, batch大小为{config.batch_size}")
print(f"test_dataloader 加载完毕, {len(test_loader)}个batch, batch大小为{config.batch_size}")


本次训练用训练集大小：9375
本次训练用验证集大小：3125
本次测试用测试集大小：2500
tarin_dataloader加载完毕, 73个batch, batch大小为128
val_dataloader  加载完毕, 25个batch, batch大小为128
test_dataloader 加载完毕, 20个batch, batch大小为128


#### 记录本次训练和测试用的数据量，还有数据样本信息

In [7]:
#查看数据加载情况
for inputs, labels in train_loader:
    inputs=inputs.to(config.device)
    labels=labels.to(config.device)
    print(inputs.shape)
    print(labels.shape)
    config.update(
        inputs_shape=inputs.shape
        )
    break
config.update(
    train_datasize=train_dataset_size,
    val_datasetsize=val_dataset_size,
    test_datasetsize=test_dataset_size,
    datasets_crop_rate=crop_rate,
    classes=train_val_dataset.classes,#原始数据集保留classes
)
# config

torch.Size([128, 3, 128, 128])
torch.Size([128])


#### 加载预训练模型

In [ ]:

def get_pretrained(hidden_size=config.hidden_size,config=config):
    '''
        获取预训练模型
        @param hidden_size: 隐藏层大小
        @param config: 配置文件
        @return: 预训练模型
    '''
    # model=net.BinaryClassificationMobileNetV3Large(out_size=len(config.classes))
    # model=net.AutoCNN(
    #     input_channels=config.inputs_shape[1],
    #     input_size=config.inputs_shape[2:],
    #     num_classes=len(config.classes),
    #     mlp_hidden_size=256
    # )
    # model=net.ResNet(
    #     num_classes=len(config.classes),
    #     input_channels=config.inputs_shape[1],
    # )#最后全局池化层压下了尺寸，不需要提供输入尺寸
    model=net.VGGNet(
        input_channels=config.inputs_shape[1],
        num_classes=len(config.classes),
        config="D",
        classifier_hidden_size=[4096,1024]
    )
    return model.to(config.device)

test_model=get_pretrained()
measurer=metrics.ModelMeasurer(test_model)
unit=1
inference_time=measurer.get_inference_time(input_shape=config.inputs_shape,unit=unit)
parameters_num=measurer.get_parameters_num()
print(f"inference_time:{inference_time} s")
print(f"parameters_num:{parameters_num}")

config.update(
    network=test_model.__class__.__name__,
    inference_time=inference_time,
    parameters_num=parameters_num,
)
config

[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']


Warm up ....:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:

bestMod=utils.BestSelector(acc=0)
train_logs=utils.Logs()
model=get_pretrained()
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss()
if config.optim == "Adam":
    optimizer = optim.Adam(model.parameters(), lr=config.lr)
else:
    optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=0.9, weight_decay=1e-4)
    
bestMod,train_logs=train_val.train_model(
            model, 
            criterion,
            optimizer, 
            train_loader,
            val_loader,
            bestMod=bestMod,
            train_logs=train_logs,
            config=config, 
            checkpoint_interval=config.checkpoint_interval,
            show_progress_interval=3
            # num_epochs=config.epochs
        )



[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 1024, 'M']


Training Epochs:   0%|          | 0/20 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.12 GiB (GPU 0; 6.00 GiB total capacity; 2.56 GiB already allocated; 960.57 MiB free; 3.59 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [10]:
print(f"{config.epochs} epoch中 最好的模型")
print(bestMod)

5 epoch中 最好的模型
acc : 0.36448 ,loss : 1.739696834185352 ,precision : 0.35245610319885845 ,recall : 0.36198650491414563 ,ap : 0.12671936742577716 ,epoch : 4 


#### 保存模型超参数和训练日志

In [11]:
saveDir=r'save_weights'
saveDir=os.path.join(
        saveDir,
        f'{bestMod.model.__class__.__name__[:10]}-acc={round(bestMod.acc,5)}-loss={round(bestMod.loss,3)}-max_epochs={config.epochs}'
)
utils.saveProcess(
    saveDir=saveDir,
    bestMod=bestMod,
    train_log=train_logs,
    config=config
)

save_weights\VGGNet-acc=0.36448-loss=1.74-max_epochs=5
{'acc': 0.36448, 'model': 'save_weights\\VGGNet-acc=0.36448-loss=1.74-max_epochs=5\\best.pth', 'loss': 1.739696834185352, 'precision': 0.35245610319885845, 'recall': 0.36198650491414563, 'ap': 0.12671936742577716, 'epoch': 4, 'checkpoints': {'checkpoint_0': 'save_weights\\VGGNet-acc=0.36448-loss=1.74-max_epochs=5\\checkpoint_0.pth', 'checkpoint_1': 'save_weights\\VGGNet-acc=0.36448-loss=1.74-max_epochs=5\\checkpoint_1.pth', 'checkpoint_2': 'save_weights\\VGGNet-acc=0.36448-loss=1.74-max_epochs=5\\checkpoint_2.pth', 'checkpoint_3': 'save_weights\\VGGNet-acc=0.36448-loss=1.74-max_epochs=5\\checkpoint_3.pth', 'checkpoint_4': 'save_weights\\VGGNet-acc=0.36448-loss=1.74-max_epochs=5\\checkpoint_4.pth'}}


#### 进行测试

In [12]:
import torch
from utils import utils,train_val
import os

# config=Config(os.path.join(dir,'config.json'))
# model=BestSelector(os.path.join(dir,'metrics.json'))
# saveDir=r'save_weights\BinaryClassificationMobileNetV3Large-acc=0.74336-loss=1.671334-max_epochs=40-1100'

Model,config,logs=utils.loadProcess(saveDir=saveDir)
metrics=train_val.validate_model(
    model=Model.model,
    val_loader=test_loader,
    device=config.device,
    only_val=True
)

metrics

模型测试中:: 100%|██████████| 20/20 [00:00<00:00, 21.65it/s]


{'Accuracy': 0.3352,
 'Precision': 0.3414599255340706,
 'Recall': 0.3299843649318467,
 'F1': 0.27500441721208524,
 'AP': 0.11960252019897313}

#### 保存数据到tensorboard

In [13]:
from utils import metrics,utils


recoder=metrics.TensorboardRecorder(
    log_dir="runs/"
)

recoder.logs_scalars(
    logs.logs,
    prefix="train"
)



#### 多个模型的训练日志展示
 

In [14]:
from utils import metrics,utils
import os
model_dirs=[
    r"D:\Desktop\深度学习\12.康复训练\My_DL_Utils\Project\save_weights\AutoCNN-acc=0.3832-loss=2.082528-max_epochs=10-1100",
    r"D:\Desktop\深度学习\12.康复训练\My_DL_Utils\Project\save_weights\BinaryClassificationMobileNetV3Large-acc=0.662-loss=1.719393-max_epochs=10-1100"
]
recorders=[]
for model_dir in model_dirs:
    Model,config,logs=utils.loadProcess(model_dir)
    dirname=os.path.basename(model_dir)
    recorder=metrics.TensorboardRecorder(log_dir=f"runs/{dirname}")
    recorders.append(recorder)
    recorder.logs_scalars(
        scalars=logs.logs,
        prefix="train"
    )



FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Desktop\\深度学习\\12.康复训练\\My_DL_Utils\\Project\\save_weights\\AutoCNN-acc=0.3832-loss=2.082528-max_epochs=10-1100\\metrics.json'

In [ ]:
def seconds_to_hms(seconds):
    hours = seconds // 3600
    remaining_seconds = seconds % 3600
    minutes = remaining_seconds // 60
    seconds = remaining_seconds % 60
    return f"{int(hours)}: {int(minutes)}: {int(seconds)}"
timestamp=(config.epochs*inference_time*(len(train_loader)+len(val_loader)))/unit
print(f"训练完成预计用时:{seconds_to_hms(timestamp)}")